# Problem 2

Use this notebook to write your code for problem 3.

In [7]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler

scale = 1;
tf_trans = 1;

def load_data(filename, skiprows = 1):
    return np.loadtxt(filename, skiprows=skiprows, delimiter=' ')

# Load the training and test data
data_train = load_data('data/training_data.txt', 1)
X_train = data_train[:, 1:]
y_train = data_train[:, 0]

#y_train = keras.utils.np_utils.to_categorical(y_train_pre,num_classes=2)

data_test = load_data('data/test_data.txt', 1)
X_test = data_test[:,:]

if tf_trans == 1:
    from sklearn.feature_extraction.text import TfidfTransformer
    tf_transformer = TfidfTransformer(use_idf=False).fit(X_train)
    X_train = tf_transformer.transform(X_train).todense()
    X_test = tf_transformer.transform(X_test).todense()

if scale == 1:
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    


## 2C - Depth vs Width for MNIST

As in problem 2, we have conveniently provided for your use code that loads, preprocesses, and deals with the uglies of the MNIST data.

In [2]:
# load MNIST data into Keras format
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from sklearn.cross_validation import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier

C:\Users\Matt\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\Matt\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import zero_one_loss
from itertools import product
from sklearn.ensemble import VotingClassifier


svc = SVC(kernel='rbf',cache_size=7000,gamma=1/2500.,verbose=1)

#parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}

#clf = GridSearchCV(svc, parameters,verbose=1,n_jobs=-1)

svc.fit(X_train,y_train)

#sorted(clf.cv_results_.keys())

#scores = cross_val_score(clf, X_train, y_train, cv=5,verbose=1)
#print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

#clf.fit(X_train, y_train) 

pred_test = svc.predict(X_train)
pred = svc.predict(X_test)





[LibSVM]

In [15]:
pred = svc.predict(X_test)

In [17]:
file = open("labels_svm2.txt","w") 
file.write("Id,Prediction\n")
for i,x in enumerate(pred):
    file.write("%i,%i\n" % (i+1,x))

In [16]:
pred

array([1., 1., 0., ..., 0., 1., 0.])

In [35]:
X_train = np.append(X_train,X_train,axis=0);
y_train = np.append(y_train,y_train,axis=0);

In [31]:
y_train.shape

(40000,)

In [ ]:
'''
n_folds = 5
skf = StratifiedKFold(y_train, n_folds=n_folds, shuffle=True)

pred = np.zeros(10000)

for i, (train, test) in enumerate(skf):
    print("Running Fold", i+1, "/", n_folds)
    clf = SGDClassifier(loss='squared_hinge', penalty='l1', alpha=0.0001, l1_ratio=0.4) 

    clf = clf.fit(X_train[train], y_train[train]) 
    #acc_pred = 1+np.sign(clf.predict(X_train[test])) 
    acc_pred = clf.predict(X_train[test])
    acc = 1-zero_one_loss(y_train[test],acc_pred)
    
    print('Test accuracy:', acc)
    
    pred += clf.predict(X_test)

y_labs = np.maximum(0.,np.sign(pred-2.5))
'''

# Augment with samples from empirical distribution of data

In [52]:
test = np.arange(0,2000)
train_pre = np.arange(2000,10000)

X_train_t = X_train[train_pre]
y_train_t = y_train[train_pre]

doc_lengths_1 = np.count_nonzero(X_train_t[np.where(y_train_t>0)],axis=1)
doc_lengths_1 = np.squeeze(np.asarray(doc_lengths_1))

word_freqs_1 = np.sum(X_train_t[np.where(y_train_t>0)],axis=0)
word_freqs_1 = word_freqs_1/float(np.sum(word_freqs_1))
word_freqs_1 = np.squeeze(np.asarray(word_freqs_1))

doc_lengths_0 = np.count_nonzero(X_train_t[np.where(y_train_t==0)],axis=1)
doc_lengths_0 = np.squeeze(np.asarray(doc_lengths_0))

word_freqs_0 = np.sum(X_train_t[np.where(y_train_t==0)],axis=0)
word_freqs_0 = np.squeeze(np.asarray(word_freqs_0))
word_freqs_0 = word_freqs_0/float(np.sum(word_freqs_0)+1)

In [53]:
np.sum(word_freqs_0)

0.9999472602013327

In [54]:
N_extra = 80000;
train = np.arange(2000,20000+N_extra)

X_train_extra = np.zeros((N_extra,1000))
y_train_extra = np.random.randint(0,2,N_extra)

for j in np.arange(0,N_extra):
    if y_train_extra[j] == 0:
        length = np.random.choice(doc_lengths_0)
        X_train_extra[j,:] = np.random.multinomial(length, word_freqs_0)
    else:
        length = np.random.choice(doc_lengths_1)
        X_train_extra[j,:] = np.random.multinomial(length, word_freqs_1)

X_train = np.append(X_train,X_train_extra,axis=0);
y_train = np.append(y_train,y_train_extra,axis=0);
#y_train = keras.utils.np_utils.to_categorical(y_train,num_classes=2)

from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train)
X_train = tf_transformer.transform(X_train)
X_test = tf_transformer.transform(X_test)


ValueError: sum(pvals[:-1]) > 1.0

In [50]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import zero_one_loss
from itertools import product
from sklearn.ensemble import VotingClassifier


#pred = np.zeros(10000)

clf = SGDClassifier(loss='hinge', penalty='l1', alpha=0.00001, l1_ratio=0.25,max_iter=100) 
clf = clf.fit(X_train[train], y_train[train]) 

acc_pred = clf.predict(X_train[test])
acc = 1-zero_one_loss(y_train[test],acc_pred)

print('Test accuracy:', acc)

#pred += clf.predict(X_test)




Test accuracy: 0.799


In [366]:
# Try a DNN on expanded data
def create_model():
    
    model = Sequential()
    
    model.add(Dense(250,input_shape=(1000,)))
    #model.add(Activation('relu'))
    #model.add(Dropout(0.3))
    
    #model.add(Dense(50))
    #model.add(Activation('relu'))
    model.add(Dropout(0.3))
    
    model.add(Dense(20))
    #model.add(Activation('relu'))
    #model.add(Dropout(0.3))
    
    #---
    #model = Sequential()
    #model.add(Dense(100,input_shape=(1000,)))
    #model.add(LeakyReLU(alpha=0.))
    #model.add(Dense(1000,input_shape=(1000,)))
    #model.add(Activation('relu'))
    #model.add(Dense(100))
    #model.add(Dropout(0.05))
    
    model.add(Dense(2))
    model.add(Activation('softmax'))

    return model
    #model.summary()
    
model = None 
model = create_model()

model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

fit = model.fit(X_train[train], y_train[train], batch_size=128, epochs=10, verbose=1)    
score = model.evaluate(X_train[test], y_train[test], verbose=0)

pred = model.predict(X_test)[:,1]

print('Test score:', score[0])
print('Test accuracy:', score[1])

y_labs = np.maximum(0.,np.sign(pred-0.5))

Epoch 1/10
98000/98000 [==============================] - 5s 47us/step - loss: 0.3184 - acc: 0.8632
Epoch 2/10
98000/98000 [==============================] - 4s 40us/step - loss: 0.2948 - acc: 0.8751
Epoch 3/10
98000/98000 [==============================] - 4s 39us/step - loss: 0.2918 - acc: 0.8766
Epoch 4/10
98000/98000 [==============================] - 4s 40us/step - loss: 0.2901 - acc: 0.8773
Epoch 5/10
98000/98000 [==============================] - 4s 39us/step - loss: 0.2891 - acc: 0.8777
Epoch 6/10
98000/98000 [==============================] - 4s 39us/step - loss: 0.2885 - acc: 0.8778
Epoch 7/10
98000/98000 [==============================] - 4s 39us/step - loss: 0.2877 - acc: 0.8784
Epoch 8/10
98000/98000 [==============================] - 4s 39us/step - loss: 0.2875 - acc: 0.8782
Epoch 9/10
98000/98000 [==============================] - 4s 39us/step - loss: 0.2871 - acc: 0.8787
Epoch 10/10
98000/98000 [==============================] - 4s 39us/step - loss: 0.2869 - acc: 0.8782

In [335]:
file = open("labels_dnn_augmented.txt","w") 
file.write("Id,Prediction\n")
for i,x in enumerate(y_labs):
    file.write("%i,%i\n" % (i+1,x))

In [346]:
X_train[train]

<95000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 2304032 stored elements in Compressed Sparse Row format>

In [364]:
y_train = keras.utils.np_utils.to_categorical(y_train,num_classes=2)